# On Assessing Your Environment
---

## Overview

Over the course of the YGL Workshop at Princeton, you will learn how we estimate, quantify, and classify and optimize energy globally and locally. A necessary aspect for this is measuring what we can as consistently and accurately as possible. 

In Tuesday's lecture we will discuss is some detail how we measure and estimate energy consumption as well as how we establish **set points** (or operational targets) around which to minimize energy consumption.

In the US, approximately 1/3rd of all energy (fuel and electricity) is consumed in maintaining a comfortable environment through heating, ventilation, and air conditions (HVAC). This is not a trivial quantity of energy. 

In this introduction, you will learn a few ways in which building conditions can be measured, and you will learn how low-cost microcontrollers are used to convert physically quantities in to digital information that can be shared, stored, and analyzed for real-time and collective decision making.


## How We Achieve Comfort

In the world of HVAC, there are four gross classes of devices

- Actuators
- Sensors
- Controllers
- Passives

**Actuators** are the elements on which most energy is spent, and these are the boilers, air conditioners, fans, space heaters etc. These devices inherently have no sense of what the environment is around them, they just convert energy to change the local environment (temperature, humidity, airflow, etc).

**Sensors** are then required to provide information that can be used to inform the actuation. Sensors are used to measure temperature, humidity, light, airflow, occupancy, etc.

The sensor and the actuator are inherently "dumb". A **controller** is used to make decisions based on the sensor reading to determine when and how the actuators should behave. The controller typically a computer program running on a thermostat or within a specialize CPU within a building.

Finally, **passives** are objects which do not sense, actuate, nor controller but rather physically bound the environment. Passives include walls, ceilings, insulation, clothes, anything that can be used to define an "inside" and "outside". Passives are critical for physically delimiting the space a sensor, an actuator and controller will work to define comfort.

## How We Sense

### A Computer Understands Switches
Just as a computer has no inherent concept comfort, it has no inherent sense temperature, humidity, light, or sound. A computer, as you might have heard, understand `0` and `1` and binary combinations thereof, and when interacting with the physical world, what defines `0` or `1` is electrical potential, or **voltage**.

At its core, a `0` or `1` is the base unit of measurement, and this choice is called a **bit**. While the world is full of important binary aspects like light and dark, cold or hot, yes or no, we are typically most comfortable between extremes. 

Computers can do nothing but remember and count, but they are really good at both, so we can chain bits together to get better levels of discretion. 

So if a one bit sensor tells me whether something 

1. Bright or 
2. Not Bright

a two-bit sensor might be able to distinguish

1. Very Bright
2. Pretty Bright
3. Sort of Bright
4. Not Bright 

We see above that bits operate in powers of 2, such that

$$ a = 2^{n}$$

where $a$ is the levels of discretion and $n$ is the number of bits chained together. So three bits provides eight levels of discretion, eight bits provides 256 levels of discretion, 12 bits provides 4096 levels of discretion, and so on.

The real world is **infinitely and infinitesimally analog**, so depending on how we wire our sensor and our bit wise sensitivity, the infinite possibilities temperature can be in reality are bound to a finite number of measurements within the computer.

### Back To Voltage

But before we can understand light, temperature and humidity from bits, we need to convert light, temperature humidity into a voltage, and then that voltage into a bit.

Starting backwards, the computer sets limits on what `0` (off) and `1` (on) are in terms of voltage, and this voltage is defined by a combination of the power that computer operates on and how the power conditioned. 

While there are a large number of operating voltage standards, a modern and ubiquitous standard is the USB minimum supply voltage of 5 V. On most small devices (e.g. phones), 5 V is the maximum potential anything within the phone sees, and the USB port provides this potential. 

But the USB port only needs to provide 5 V +/- 5%. This is just fine for power, but it can make sensor readings a bit challenging if absolute voltage is required. For this reason, and reasons of chip standardization, on modern small devices the sort of 5 V power line is stepped down to a very firm 3.3 V, so 3.3 V defines our `1`. Our `0` is the common ground of the USB device. 

Based on our binning above, for a 1 bit sensor, anything that is north of 1.6 V (3.3V/2) will generally be binned "on", and anything below this is generally binned off.

Luckily, the devices we will be using to measure things have 12 bit analog to digital (ADC), so 0 is 0V and 3.3V is 4095. 


